In [245]:
import pandas as pd
from simstring.feature_extractor.character_ngram import CharacterNgramFeatureExtractor
from simstring.measure.cosine import CosineMeasure
from simstring.database.dict import DictDatabase
from simstring.searcher import Searcher
from simstring.feature_extractor.word_ngram import WordNgramFeatureExtractor
from simstring.measure.jaccard import JaccardMeasure
from simstring.database.mongo import MongoDatabase
from simstring.searcher import Searcher
import numpy as np
import re
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)

In [246]:
pwd

'/Users/daisukeyagyu/Desktop/UChicago/Courses/Capstone/00_Implementation/Delete_AfterProject/DataPreparation'

# Spec

In [393]:
# df_spec = pd.read_excel('./Combination_Data/SPEC/Spec_combined.xlsx')
df_spec = pd.read_csv('./Combination_Data/SPEC/Spec_combined.csv')

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/3665924712.py:2: DtypeWarning: Columns (15,20,22,23,32,37,39,40,44,47,54,62,63,74,77,85,86,87,96,98,125,141,143,145,152,162,163,166,169,171,175,184,186,208,209,212,215,218,221,222,223,228,230,231,234,236,243,245,246,247,248,249,250,252,253,255,257,258,266,270,277,279,289,290,295,304,307,311,313,314,321,324,325,335,337,338,339,340,341,343,344,356,361,363,412,417,422,425,430,431,434,444,448,450,453,458,460,461,463,466,467,470,472,473,474,478,479,480,484,485,486,487,488,489,492,494,498,500,501,502,503,506,509,511,513,515,516,517,518,519,520,522,523,524,525,526,527,528,531,533,535,536,538,540,541,543,545,551,552,553,557,558,560,561,562,568,569,570,571,572,573,574,576,581,583,584,585,590,591,592,593,595,596,597,602,603,604,606,608,616,617,618,619,621,633,634,639,645,646,649,658,659,660,661,662,667,670,671,672,673,674,679,682,683,684,685,686,690,691,692,693,695,696,697,700,706,708,709,711,719,720,722,723,725,728,730,735,785,803

In [249]:
# df_spec.to_csv("Spec_combined.csv", index=False)

In [3]:
df_spec['Version state'].value_counts()

price change                                            19140
model year change                                       10385
option change                                            6957
out going                                                3701
option change:price change                               1662
new version                                              1603
price change:option change                               1456
face lift:model year change                               750
new model                                                 664
model year change:new model                               534
model year change:face lift                               471
new model:model year change                               219
specification change                                      184
price change:specification change                          95
face lift                                                  74
new model:new version                                      46
new vers

This analysis does not include option contents, so ignore the option changes.

In [394]:
out_of_scope = ['option change', 'out going', 'option change:price change', 'price change:option change',
'option change:specification change', 'price change:option change:specification change']

In [395]:
df_spec = df_spec[~df_spec['Version state'].isin(out_of_scope)]

In [396]:
df_spec = df_spec.add_prefix('SPEC_')

# RV table

## Create Unique Keys

In [397]:
# df_rv = pd.read_excel('./Combination_Data/RV/RV_combined.xlsx')
df_rv = pd.read_csv('./Combination_Data/RV/RV_combined.csv')

In [255]:
# df_rv.to_csv('./Combination_Data/RV/RV_combined.csv', index=False)

In [256]:
df_rv.head()

,EditionMonth,EditionYear,Unnamed: 2,ModelYear,Make,Model,Model#,ModelDesc,MSRP,12mo,24mo,36mo,48mo,60mo,AlgMake,AlgModel,AlgStyle,MRM
0,9,2022,2023VOLVOXC90 AWD4dr Wgn T8 Ultimate w/6 Pass,2023,VOLVO,XC90 AWD,XC90T8UAWD6,4dr Wgn T8 Ultimate w/6 Pass,NaN,81,66,52,44,35,860,400,162,81900.0
1,11,2022,2023VOLVOXC90 AWD4dr Wgn T8 Ultimate w/6 Pass,2023,VOLVO,XC90 AWD,XC90T8UAWD6,4dr Wgn T8 Ultimate w/6 Pass,NaN,79,65,51,43,33,860,400,162,81900.0
2,3,2023,2023VOLVOXC90 AWD4dr Wgn T8 Ultimate w/6 Pass,2023,VOLVO,XC90 AWD,XC90T8UAWD6,4dr Wgn T8 Ultimate w/6 Pass,NaN,76,61,48,41,32,860,400,162,81900.0
3,9,2022,2023VOLVOXC90 AWD4dr Wgn T8 Ultimate,2023,VOLVO,XC90 AWD,XC90T8U*AWD7,4dr Wgn T8 Ultimate,NaN,81,66,53,44,35,860,400,161,81300.0
4,11,2022,2023VOLVOXC90 AWD4dr Wgn T8 Ultimate,2023,VOLVO,XC90 AWD,XC90T8U*AWD7,4dr Wgn T8 Ultimate,NaN,79,65,51,43,34,860,400,161,81300.0


In [398]:
df_rv['UID_RV'] = df_rv['ModelYear'].astype(str) + ' '+ df_rv['Make'] + ' ' + df_rv['Model'] +' '+ df_rv['ModelDesc']

In [399]:
df_rv['UID_woYear_RV'] = df_rv['Make'] + ' ' + df_rv['Model'] + ' '+ df_rv['ModelDesc']

In [400]:
len(df_rv['UID_woYear_RV'].unique())

9275

In [401]:
len(df_spec['SPEC_Version name'].unique())

4639

In [402]:
df_spec['SPEC_Version name'] = df_spec['SPEC_Version name'].str.replace('ｮ', '').replace('ｺ', '')


In [403]:
# df_spec['RV_UID_Spec'] = df_spec['Model year'].astype(str) + ' '+ df_spec['Make'] + ' ' + df_spec['Model'] + ' ' + df_spec['Version name'] + ' ' + df_spec['Body type']
df_spec['RV_UID_woYear_Spec'] = df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model'] + ' ' + df_spec['SPEC_Version name'] +' ' + df_spec['SPEC_Body type']

## Join tables

### RV table aggregation for joining

In [404]:
df_rv = df_rv[['UID_RV','MSRP', '12mo', '24mo', '36mo', '48mo','60mo']]

In [405]:
df_rv['12mo'] = df_rv.apply(lambda x: (x['12mo'] / x['MSRP'])*100 if x['12mo']>120 else x['12mo'], axis=1)
df_rv['24mo'] = df_rv.apply(lambda x: (x['24mo'] / x['MSRP'])*100 if x['24mo']>120 else x['24mo'], axis=1)
df_rv['36mo'] = df_rv.apply(lambda x: (x['36mo'] / x['MSRP'])*100 if x['36mo']>120 else x['36mo'], axis=1)
df_rv['48mo'] = df_rv.apply(lambda x: (x['48mo'] / x['MSRP'])*100 if x['48mo']>120 else x['48mo'], axis=1)
df_rv['60mo'] = df_rv.apply(lambda x: (x['60mo'] / x['MSRP'])*100 if x['60mo']>120 else x['60mo'], axis=1)

In [406]:
df_rv = df_rv.groupby('UID_RV').mean()[['12mo', '24mo', '36mo', '48mo','60mo']].reset_index()

In [407]:
df_rv.head()

,UID_RV,12mo,24mo,36mo,48mo,60mo
0,2010 ACURA MDX 4dr Wgn,67.0,58.0,49.0,42.0,35.0
1,2010 ACURA MDX 4dr Wgn Advance,60.0,51.0,44.0,37.0,31.0
2,2010 ACURA MDX 4dr Wgn Advance w/RES,59.0,51.0,43.0,37.0,31.0
3,2010 ACURA MDX 4dr Wgn Tech,65.0,56.0,48.0,41.0,34.0
4,2010 ACURA MDX 4dr Wgn Tech w/RES,64.0,55.0,47.0,40.0,34.0


### Combination

In [408]:
spec_rv_jointable = pd.read_excel('./Combination_Data/RV/spec_rv_jointable.xlsx')

In [409]:
spec_rv_jointable

,RV_UID_woYear_Spec,RV_UID_woYear_RV
0,ACURA MDX 3.5 AUTO sport utility vehicle,ACURA MDX 2WD 4dr Wgn
1,ACURA MDX 3.5 4WD AUTO sport utility vehicle,ACURA MDX AWD 4dr Wgn
2,ACURA MDX 3.5 AUTO TECHNOLOGY PACKAGE sport ut...,ACURA MDX 2WD 4dr Wgn Tech
3,ACURA MDX 3.5 AUTO TECH & ENT PACKAGE sport ut...,ACURA MDX 2WD 4dr Wgn Tech
4,ACURA MDX 3.5 AUTO 4WD TECHNOLOGY PACKAGE spor...,ACURA MDX AWD 4dr Wgn Tech
...,...,...
8346,VOLKSWAGEN ID.4 EV 1ST EDITION sport utility v...,VOLKSWAGEN ID.4 4dr Wgn 1st Edition
8347,VOLKSWAGEN ID.4 EV PRO 4WD sport utility vehicle,VOLKSWAGEN ID.4 4dr Wgn Pro AWD
8348,VOLKSWAGEN ID.4 EV PRO S 4WD sport utility veh...,VOLKSWAGEN ID.4 4dr Wgn Pro S AWD
8349,VOLKSWAGEN ID.4 EV PRO S sport utility vehicle,VOLKSWAGEN ID.4 4dr Wgn Pro S


In [410]:
df_rv = df_rv.add_prefix('RV_')

In [411]:
df_spec = pd.merge(df_spec, spec_rv_jointable, on='RV_UID_woYear_Spec', how='left')

In [412]:
df_spec['RV_UID_RV'] = df_spec['SPEC_Model year'].astype(str) + ' ' + df_spec['RV_UID_woYear_RV'] 

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/3170493136.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['RV_UID_RV'] = df_spec['SPEC_Model year'].astype(str) + ' ' + df_spec['RV_UID_woYear_RV']


In [413]:
df_spec = pd.merge(df_spec, df_rv, on='RV_UID_RV', how='left')

### Ratio of unmatch columns

In [414]:
sum(df_spec['RV_12mo'].isna()) / len(df_spec)

0.09338747099767981

In [415]:
df_spec.columns

Index(['SPEC_Name', 'SPEC_Unique Identity', 'SPEC_Data date',
       'SPEC_Date of change', 'SPEC_Conclude date', 'SPEC_Version state',
       'SPEC_Data status', 'SPEC_Data completeness', 'SPEC_msrp',
       'SPEC_colour & trim status',
       ...
       'SPEC_dual rear wheel', 'SPEC_super single', 'RV_UID_woYear_Spec',
       'RV_UID_woYear_RV', 'RV_UID_RV', 'RV_12mo', 'RV_24mo', 'RV_36mo',
       'RV_48mo', 'RV_60mo'],
      dtype='object', length=2691)

In [416]:
df_spec = df_spec.drop(['RV_UID_woYear_Spec', 'RV_UID_woYear_RV', 'RV_UID_RV'], axis=1)

# OaO Table

In [417]:
df_oao =  pd.read_excel('./Combination_Data/OaO/OaO_combined_brand.xlsx')

In [418]:
df_oao

,Date_CY FY_Date Type,Corporate,Line split,Model,Nissan group,Score (latest),Score,Score.1,Brand
0,FY 14,VW,NaN,Passat,NaN,NaN,0.116103,0.116103,NaN
1,FY 15,VW,NaN,Passat,NaN,NaN,0.115625,0.115625,NaN
2,FY 16,VW,NaN,Passat,NaN,NaN,0.091099,0.091099,NaN
3,FY 17,VW,NaN,Passat,NaN,NaN,0.091660,0.091660,NaN
4,FY 18,VW,NaN,Passat,NaN,NaN,0.081817,0.081817,NaN
...,...,...,...,...,...,...,...,...,...
2812,FY 18,Chevrolet,NaN,Avalanche,NaN,NaN,NaN,NaN,NaN
2813,FY 19,Chevrolet,NaN,Avalanche,NaN,NaN,NaN,NaN,NaN
2814,FY 20,Chevrolet,NaN,Avalanche,NaN,NaN,NaN,NaN,NaN
2815,FY 21,Chevrolet,NaN,Avalanche,NaN,NaN,NaN,NaN,NaN


## Create unique keys

In [419]:
df_spec['SPEC_DataYear'] = pd.to_datetime(df_spec['SPEC_Data date']).dt.year

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/353249121.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['SPEC_DataYear'] = pd.to_datetime(df_spec['SPEC_Data date']).dt.year


In [420]:
df_oao = df_oao[df_oao['Brand']==1]

In [421]:
def fytoyear(x):
    return '20'+ x.split('FY ')[1] 

In [422]:
df_oao['Year'] = df_oao['Date_CY FY_Date Type'].apply(lambda x: fytoyear(x))

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/4184590986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oao['Year'] = df_oao['Date_CY FY_Date Type'].apply(lambda x: fytoyear(x))


In [423]:
df_oao = df_oao[['Year', 'Corporate', 'Score']]

In [424]:
df_oao['UID_OAO'] = df_oao['Year'] +' '+ df_oao['Corporate']

## Join Tables

In [425]:
spec_oao_jointable = pd.read_csv('./Combination_Data/OaO/spec_oao_jointable.csv')

In [426]:
spec_oao_jointable.head()

,Make,Corporate
0,ACURA,Acura
1,AUDI,Audi
2,BMW,BMW
3,CADILLAC,Cadillac
4,CHEVROLET,Chevrolet


### Combination

In [427]:
spec_oao_jointable = spec_oao_jointable.rename(columns={"Make":"SPEC_Make"})

In [428]:
df_spec = pd.merge(df_spec, spec_oao_jointable, on='SPEC_Make', how='left')

In [429]:
df_spec['OAO_UID_OAO'] = df_spec['SPEC_DataYear'].astype(str) + ' ' + df_spec['Corporate']

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/1507792015.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['OAO_UID_OAO'] = df_spec['SPEC_DataYear'].astype(str) + ' ' + df_spec['Corporate']


In [430]:
df_oao = df_oao.add_prefix('OAO_')

In [431]:
df_spec = pd.merge(df_spec, df_oao, on='OAO_UID_OAO', how='left')

### Ratio of unmatch rows

In [432]:
sum(df_spec['OAO_Score'].isna()) / len(df_spec)

0.03935614849187935

In [433]:
df_spec = df_spec.drop(['SPEC_DataYear',
       'Corporate', 'OAO_UID_OAO', 'OAO_Year', 'OAO_Corporate',], axis=1)

# Edmunds

In [434]:
df_edmunds = pd.read_csv('./Combination_Data/Edmunds/df_inference.csv')

In [435]:
df_edmunds.columns

Index(['UID', 'Message', 'Intel Product', 'Star Rating', 'Variant',
       'ConversationId', 'ModelYear', 'Brand', 'Model', 'sentence_list',
       'ADAS_0_prob', 'ADAS_1_prob', 'Affordability_0_prob',
       'Affordability_1_prob', 'ComfortQuietness_0_prob',
       'ComfortQuietness_1_prob', 'FuelEfficiency_0_prob',
       'FuelEfficiency_1_prob', 'Infortainment_0_prob', 'Infortainment_1_prob',
       'ExteriorDesign_0_prob', 'ExteriorDesign_1_prob',
       'InteriorDesign_0_prob', 'InteriorDesign_1_prob', 'Safety_0_prob',
       'Safety_1_prob', 'Utility_0_prob', 'Utility_1_prob',
       'DP_Ridecomfort_0_prob', 'DP_Ridecomfort_1_prob', 'sentiment_nega_prob',
       'sentiment_posi_prob'],
      dtype='object')

In [436]:
category_dct = {'ADAS':0.9, 'Affordability':0.97, 'ComfortQuietness':0.85, 
                 'DP_Ridecomfort':0.85, 'ExteriorDesign':0.95,'FuelEfficiency':0.88,
                 'Infortainment':0.93, 'InteriorDesign':0.95,'Safety':0.9, 'Utility':0.99}

In [437]:
for category in category_dct.keys():
    df_edmunds[category] = df_edmunds[category+'_1_prob'].apply(lambda x: 1 if x > category_dct[category] else 0)

In [438]:
df_edmunds['Sentiment'] = df_edmunds.apply(lambda x: 1 if x['sentiment_posi_prob'] > 0.99 else -1 if x['sentiment_nega_prob']>0.99 else 0, axis=1)

In [439]:
df_edmunds = df_edmunds[['UID', 'Message', 'Intel Product', 'Star Rating', 'Variant',
       'ConversationId', 'ModelYear', 'Brand', 'Model', 'sentence_list', 'ADAS', 'Affordability', 
       'ComfortQuietness', 'DP_Ridecomfort', 'ExteriorDesign', 'FuelEfficiency', 'Infortainment', 
       'InteriorDesign', 'Safety', 'Utility', 'Sentiment']]

In [440]:
df_edmunds['Sentiment'].value_counts()

 1    218621
-1    191458
 0    106089
Name: Sentiment, dtype: int64

In [441]:
for category in category_list:
    df_edmunds[category + "sentiscore"] = df_edmunds[category] * df_edmunds['Sentiment']

In [442]:
df_edmunds_bymention = df_edmunds[['Brand', 'Model','ModelYear', 'ConversationId','Star Rating','ADASsentiscore',
       'Affordabilitysentiscore', 'ComfortQuietnesssentiscore',
       'DP_Ridecomfortsentiscore', 'ExteriorDesignsentiscore',
       'FuelEfficiencysentiscore', 'Infortainmentsentiscore',
       'InteriorDesignsentiscore',  'Safetysentiscore',
       'Utilitysentiscore']].groupby(['Brand', 'Model','ModelYear', 'ConversationId']).mean()

In [443]:
df_edmunds_byModels = df_edmunds_bymention.groupby(['Brand', 'Model','ModelYear']).mean()

## Create unique IDs 

In [444]:
df_spec['Make_Model_spec'] =  df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model'] 

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/2723585430.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['Make_Model_spec'] =  df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model']


In [445]:
df_edmunds_byModels = df_edmunds_byModels.reset_index()

In [446]:
df_edmunds_byModels['Make_Model_Edmunds'] = + df_edmunds_byModels['Brand'] + ' ' + df_edmunds_byModels['Model']

In [447]:
df_edmunds_byModels['Make_Model_Edmunds'] = df_edmunds_byModels['Make_Model_Edmunds'].str.replace('Ford Transit Passenger Van','Ford Transit')\
.replace('Ford Transit Van', 'Ford Transit')\
.replace('Ford Transit Wagon', 'Ford Transit')\
.replace('Chevrolet Express Cargo', 'Chevrolet Express')\
.replace('Acura NSX Reviews; Specs and Price', 'Acura NSX')\
.replace('Acura TLX A-Spec\u003Csup&gt;\u003C/sup&gt;', 'Acura TLX')\
.replace('Audi TT RS', 'Audi TT')\
.replace('Audi TTS', 'Audi TT')\
.replace('BMW X5 M', 'BMW X5')\
.replace('Ford Edge Prices; Reviews and Pictures', 'Ford Edge')\
.replace('Ford Escape Hybrid', 'Ford Escape')\
.replace('Ford F-150 Goes Electric', 'Ford F-150')\
.replace('ford f150', 'Ford Focus')\
.replace('Ford Fusion Hybrid', 'Ford Fusion')\
.replace('Ford Ranger May Add Raptor Model', 'Ford Ranger')\
.replace('GMC Sierra 1500 Denali', 'GMC Sierra 1500')\
.replace('GMC Yukon XL', 'GMC Yukon')\
.replace('Honda Accord Hybrid', 'Honda Accord')\
.replace('Honda CR-V Hybrid', 'Honda CR-V')\
.replace('Hyundai Elantra GT', 'Hyundai Elantra')\
.replace('Hyundai Kona Electric', 'Hyundai Kona')\
.replace('Hyundai Sonata Hybrid', 'Hyundai Sonata')\
.replace('Hyundai Tucson Takes Compact SUV Class in Bold New Direction', 'Hyundai Tucson')\
.replace('Lexus IS 250', 'Lexus IS')\
.replace('Lexus IS 300', 'Lexus IS')\
.replace('Lexus IS 350', 'Lexus IS')\
.replace('Lexus IS 500 Is a V8-Driven Swan Song for Long-Running; Third-Gen IS', 'Lexus IS')\
.replace('Lexus LC 500', 'Lexus LC')\
.replace('Lexus LC 500 Convertible', 'Lexus LC')\
.replace('Lexus LX 570', 'Lexus LX')\
.replace('Lexus LX 570 Two-Row', 'Lexus LX')\
.replace('Lexus NX 200t', 'Lexus NX')\
.replace('Lexus NX 300', 'Lexus NX')\
.replace('Lexus NX 300h', 'Lexus NX')\
.replace('Lexus NX 350', 'Lexus NX')\
.replace('Lexus RC 300', 'Lexus RC')\
.replace('Lexus RC 350', 'Lexus RC')\
.replace('Lexus RC F', 'Lexus RX')\
.replace('Lexus RX 350', 'Lexus RX')\
.replace('Lexus RX 350L', 'Lexus RX')\
.replace('Lexus RX 450h', 'Lexus RX')\
.replace('Lexus RX 450hL', 'Lexus RX')\
.replace('Mercedes-Benz C-Class Extends Luxury Appeal for Less Dough', 'Mercedes-Benz C-Class')\
.replace('Mercedes-Benz GLE', 'Mercedes-Benz GLE-Class')\
.replace('Mercedes-Benz GLS', 'Mercedes-Benz GLS-Class')\
.replace('Nissan Maxima Likely to Return Unchanged', 'Nissan Maxima')\
.replace('Nissan Murano Likely to Return Unchanged', 'Nissan Murano')\
.replace('Nissan NV Cargo', 'Nissan NV')\
.replace('Nissan NV Passenger', 'Nissan NV')\
.replace('Nissan Pathfinder: Blocky New Styling Masks a Whole Lot of Carryover', 'Nissan Pathfinder')\
.replace('Nissan Versa Sedan', 'Nissan Versa')\
.replace('Ram Pro Master Window Van', 'Ram Pro Master')\
.replace('Ram ProMaster', 'Ram Pro Master')\
.replace('Ram ProMaster Cargo Van', 'Ram Pro Master')\
.replace('Toyota Avalon Hybrid', 'Toyota Avalon')\
.replace('Toyota Camry Hybrid', 'Toyota Camry')\
.replace('Toyota Corolla Hatchback', 'Toyota Corolla')\
.replace('Toyota Corolla Hybrid', 'Toyota Corolla')\
.replace('Toyota Highlander Hybrid', 'Toyota Highlander')\
.replace('Toyota RAV4 Hybrid', 'Toyota RAV4')\
.replace('Toyota RAV4 Prime', 'Toyota RAV4')\
.replace('Toyota Yaris Hatchback', 'Toyota Yaris')


In [448]:
df_edmunds_byModels['UID_Edmunds'] = df_edmunds_byModels['ModelYear'].astype('str') + ' ' + df_edmunds_byModels['Make_Model_Edmunds'] 

## Combination

In [449]:
spec_edmunds_jointable = pd.read_csv('./Combination_Data/Edmunds/spec_edmunds_jointable.csv')

In [450]:
spec_edmunds_jointable

,Make_Model_spec,Make_Model_Edmunds
0,ACURA MDX,Acura MDX
1,ACURA NSX,Acura NSX
2,ACURA RDX,Acura RDX
3,ACURA RLX,Acura RLX
4,ACURA TLX,Acura TLX
...,...,...
159,TOYOTA TACOMA,Toyota Tacoma
160,TOYOTA TUNDRA,Toyota Tundra
161,TOYOTA YARIS,Toyota Yaris
162,TOYOTA YARIS IA,Toyota Yaris


In [451]:
df_spec = pd.merge(df_spec, spec_edmunds_jointable, on='Make_Model_spec', how='left')

In [452]:
df_spec['Edmunds_UID_Edmunds'] = df_spec['SPEC_Model year'].astype(str) + ' ' + df_spec['Make_Model_Edmunds'] 

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/3355971129.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['Edmunds_UID_Edmunds'] = df_spec['SPEC_Model year'].astype(str) + ' ' + df_spec['Make_Model_Edmunds']


In [453]:
df_edmunds_byModels = df_edmunds_byModels[['Star Rating', 'ADASsentiscore',
       'Affordabilitysentiscore', 'ComfortQuietnesssentiscore',
       'DP_Ridecomfortsentiscore', 'ExteriorDesignsentiscore',
       'FuelEfficiencysentiscore', 'Infortainmentsentiscore',
       'InteriorDesignsentiscore',  'Safetysentiscore',
       'Utilitysentiscore', 'UID_Edmunds']]

In [454]:
df_edmunds_byModels = df_edmunds_byModels.add_prefix('Edmunds_')

In [455]:
df_spec = pd.merge(df_spec, df_edmunds_byModels, on='Edmunds_UID_Edmunds', how='left')

In [456]:
df_spec = df_spec.drop(['Make_Model_spec', 'Make_Model_Edmunds', 'Edmunds_UID_Edmunds'], axis=1)

### Ratio unmatch rows

In [457]:
df_spec.head(1)

,SPEC_Name,SPEC_Unique Identity,SPEC_Data date,SPEC_Date of change,SPEC_Conclude date,SPEC_Version state,SPEC_Data status,SPEC_Data completeness,SPEC_msrp,SPEC_colour & trim status,SPEC_key leasing items,SPEC_option content status,SPEC_option prices status,SPEC_prices status,SPEC_specs status,SPEC_incentive status,SPEC_Model year,SPEC_Production Year,SPEC_Country,SPEC_Make,SPEC_Global equivalent make,SPEC_Model,SPEC_Global equivalent model,SPEC_Model descriptor,SPEC_Model generation,SPEC_Version name,SPEC_Engine level,SPEC_Special edition,SPEC_Limited edition,SPEC_Special edition base UID,SPEC_Trim level,SPEC_trim classification,SPEC_trim code,SPEC_Pricing descriptor,SPEC_Transmission type,SPEC_Number of doors,SPEC_Body type,SPEC_cab type,SPEC_tilting cab,SPEC_body length,SPEC_duty rating,SPEC_Powertrain type,SPEC_msrp2,SPEC_type,SPEC_roof type,SPEC_wheelbase type,SPEC_Driver location,SPEC_platform code,SPEC_Driven wheels,SPEC_Liters,SPEC_Maximum power kW,SPEC_Maximum power hp/PS,SPEC_Fiscal HP,SPEC_Fuel type,SPEC_Compressor,SPEC_Seating capacity,SPEC_Price currency,SPEC_Retail price,SPEC_Base price,SPEC_Country specific price,SPEC_Retail price including delivery charge,SPEC_Transaction price,SPEC_Net prices,SPEC_Regional prices,SPEC_msrp3,SPEC_language,SPEC_name of region,SPEC_Indicative prices only,SPEC_Cash/Net price,SPEC_Finance price,SPEC_percentage discount,SPEC_option percentage discount,SPEC_monthly leasing cost,SPEC_Source of price list,SPEC_Latest price list number,SPEC_Latest price list date,SPEC_Previous model name,SPEC_Previous version name,SPEC_Date version was linked,SPEC_Version id,SPEC_JATOCode1,SPEC_JATOCode2,SPEC_Vehicle type,SPEC_Registration type,SPEC_Country of assembly,SPEC_VIN,SPEC_Manufacturer's code,SPEC_Local manufacturer's code,SPEC_Manufacturer group,SPEC_Manufacturer,SPEC_Make group,SPEC_Model group,SPEC_Make continent group,SPEC_model range,SPEC_homologation standard,SPEC_homologation class,SPEC_Purchase condition,SPEC_Plant of assembly,SPEC_Template version,SPEC_ACRISS code,SPEC_Local language,SPEC_msrp4,SPEC_language5,SPEC_manufacturer group6,SPEC_manufacturer7,SPEC_global make name,SPEC_make8,SPEC_model9,SPEC_version name10,SPEC_trim level11,SPEC_Date introduced,SPEC_msrp12,SPEC_Number of doors introduced,SPEC_Body type introduced,SPEC_Version introduced,SPEC_Model introduced,SPEC_Drive introduced,SPEC_Engine introduced,SPEC_Transmission introduced,SPEC_Trim introduced,SPEC_Model Year Fraction,SPEC_facelift generation,SPEC_Make code,SPEC_Model code,SPEC_Version availability,SPEC_showroom arrival date,SPEC_Premark segment,SPEC_JATO global segment,SPEC_JATO regional segment,SPEC_Audio preparation,SPEC_msrp13,SPEC_Speakers,SPEC_msrp14,SPEC_number of,SPEC_front number of,SPEC_rear number of,SPEC_middle number of,SPEC_centre number of,SPEC_brand name,SPEC_subwoofer,SPEC_tweeters,SPEC_4D enhancement,SPEC_surround sound,SPEC_Antenna,SPEC_msrp15,SPEC_type16,SPEC_Audio system,SPEC_msrp17,SPEC_brand name18,SPEC_radio,SPEC_in-dash CD,SPEC_RDS,SPEC_CD player reads MP3,SPEC_digital media card,SPEC_digital radio,SPEC_satellite radio,SPEC_satellite radio months of subscription,SPEC_wireless headphones,SPEC_in-dash DVD,SPEC_internet radio,SPEC_internet radio months of subscription,SPEC_touch screen,SPEC_multi-touch screen,SPEC_audio player reads MP3,SPEC_system power (W),SPEC_colour screen,SPEC_Audio zones,SPEC_msrp19,SPEC_number of zones,SPEC_Conversation enhancement,SPEC_msrp20,SPEC_Electronic noise reduction,SPEC_msrp21,SPEC_Remote audio controls,SPEC_msrp22,SPEC_type23,SPEC_Rear separate audio control,SPEC_msrp24,SPEC_Amplifier,SPEC_msrp25,SPEC_MP3 player,SPEC_msrp26,SPEC_Disc player,SPEC_msrp27,SPEC_location,SPEC_capacity,SPEC_media type,SPEC_Connection to ext.entertainment devices,SPEC_msrp28,SPEC_game console front,SPEC_mp3 devices,SPEC_wma devices,SPEC_ipod front,SPEC_AUX jack front,SPEC_USB connection front,SPEC_total number of USB connections front,SPEC_HDMI connection front,SPEC_HDMI connection rea

In [458]:
sum(df_spec['Edmunds_ComfortQuietnesssentiscore'].isna()) / len(df_spec)

0.2471418787208261

# Segment

In [459]:
df_segment = pd.read_excel('./Combination_Data/Segment/SegmentTable.xlsx')

In [460]:
df_segment

,Make_Model_spec,Competing_Nissan_Model
0,ACURA MDX,Competitors: QX60
1,ACURA NSX,Competitors: GT-R
2,ACURA RDX,Competitors: QX55
3,ACURA RLX,Competitors: Q70
4,ACURA TLX,Competitors: MAXIMA
...,...,...
159,TOYOTA TACOMA,Competitors: FRONTIER
160,TOYOTA TUNDRA,Competitors: TITAN
161,TOYOTA YARIS,Competitors: VERSA NOTE
162,TOYOTA YARIS IA,Competitors: VERSA


In [461]:
df_spec['Make_Model_spec'] = df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model']

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/2905276647.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['Make_Model_spec'] = df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model']


In [462]:
df_spec = pd.merge(df_spec, df_segment, on='Make_Model_spec', how='left')

In [463]:
segment_name = pd.read_excel('./Combination_Data/Segment/Segment_Name.xlsx')

In [464]:
segment_name = segment_name.rename(columns={'Segment_CompetingNissanModel': 'Competing_Nissan_Model'})

In [465]:
df_spec = pd.merge(df_spec, segment_name, on='Competing_Nissan_Model', how='left')

In [466]:
len(df_spec[df_spec['Competing_Nissan_Model'].isna()]) / len(df_spec)

0.005005005005005005

In [467]:
df_spec.drop(['Make_Model_spec'],axis=1, inplace=True)

In [468]:
df_spec = df_spec.rename(columns={'Competing_Nissan_Model':'Segment_Competing_Nissan_Model'})

# Volume

In [469]:
df_volume = pd.read_csv('./Combination_Data/Volume/VolumebyMY.csv')

In [470]:
df_volume

,Make,Make:Model,Model Year,Version Name,Unique Identity,Volume
0,ABARTH,ABARTH:124 SPIDER,2018,1.4,7732180.0,933
1,ABARTH,ABARTH:124 SPIDER,2018,1.4 2 YEAR ANNIVERSARY,7541359.0,100
2,ABARTH,ABARTH:124 SPIDER,2018,1.4 AUTO,7732179.0,214
3,ABARTH,ABARTH:124 SPIDER,2018,1.4 TURBO MULTIAIR,7889654.0,164
4,ABARTH,ABARTH:124 SPIDER,2018,1.4 TURBO MULTIAIR,7743695.0,213
...,...,...,...,...,...,...
723715,ZXAUTO,ZXAUTO:GRANDTIGER TUV,2014,2.5 LUXURY 2WD,7111693.0,970
723716,ZXAUTO,ZXAUTO:GRANDTIGER TUV,2014,2.5 LUXURY 4WD,7111692.0,970
723717,ZXAUTO,ZXAUTO:GRANDTIGER TUV,2014,2.8 ELITE 4JB1 2WD,7449448.0,967
723718,ZXAUTO,ZXAUTO:GRANDTIGER TUV,2014,2.8 ELITE 4JB1 4WD,7327763.0,966


In [471]:
df_volume['Unique Identity'].fillna(0, inplace=True)
df_volume['Unique Identity'] = df_volume['Unique Identity'].astype(int).astype(str)
df_spec['SPEC_Unique Identity'] = df_spec['SPEC_Unique Identity'].astype(int).astype(str)

In [472]:
df_volume['year_uid'] = df_volume['Model Year'].astype(str) + '_' + df_volume['Unique Identity']

In [473]:
df_spec['year_uid'] = df_spec['SPEC_Model year'].astype(str) + '_' + df_spec['SPEC_Unique Identity']

In [474]:
df_volume_gb = df_volume.groupby(['Unique Identity','year_uid']).sum().reset_index()

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/1935317843.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_volume_gb = df_volume.groupby(['Unique Identity','year_uid']).sum().reset_index()


In [475]:
df_spec = pd.merge(df_spec, df_volume_gb[['year_uid', 'Volume']], on='year_uid',how='left')

### Ratio of not match rows

In [476]:
len(df_spec[df_spec['Volume'].isna()]) / len(df_spec)

0.06980664875401718

In [477]:
df_spec.drop('year_uid', axis=1, inplace=True)

## To combine price change depending on the available periods

In [478]:
df_spec['MY_Make_Model_Version'] = df_spec['SPEC_Model year'].astype(str) +  ' ' + df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model'] + ' ' + df_spec['SPEC_Version name']

In [479]:
df_spec = df_spec.sort_values(by=['SPEC_Make', 'SPEC_Model', 'SPEC_Model year', 'SPEC_Version name', 'SPEC_Data date'], ascending=[True, True, True, True, True])

In [480]:
df_spec['SPEC_Data date'] = pd.to_datetime(df_spec['SPEC_Data date'])

In [481]:
df_spec['start_date'] = df_spec['SPEC_Data date']

In [482]:
df_spec['end_date'] = df_spec.groupby('MY_Make_Model_Version')['SPEC_Data date'].shift(-1)

In [483]:
df_spec.head()

SPEC_Name SPEC_Unique Identity SPEC_Data date SPEC_Date of change  \
1    Table2               509489     2013-06-20          2013-06-20   
0    Table2              7208213     2013-06-20          2013-06-20   
7    Table2              7715230     2013-06-20          2013-06-20   
5    Table2              7358828     2013-06-20          2013-06-20   
4    Table2              7358827     2013-06-20          2013-06-20   

    SPEC_Conclude date           SPEC_Version state         SPEC_Data status  \
1  2014-06-02 00:00:00  model year change:new model               full specs   
0  2014-06-02 00:00:00        new model:new version               full specs   
7  2014-06-02 00:00:00  model year change:new model               full specs   
5  2014-06-02 00:00:00  model year change:new model               full specs   
4  2014-06-02 00:00:00  model year change:new model  full specs & inspection   

  SPEC_Data completeness  SPEC_msrp SPEC_colour & trim status  \
1               standard        NaN                up to date   
0               standard        NaN                up to date   
7               standard        NaN                up to date   
5               standard        NaN                up to date   
4               standard        NaN                up to date   

  SPEC_key leasing items SPEC_option content status SPEC_option prices status  \
1             up to date                 up to date                up to date   
0             up to date                 up to date                up to date   
7             up to date                 up to date                up to date   
5             up to date                 up to date                up to date   
4             up to date                 up to date                up to date   

  SPEC_prices status                    SPEC_specs status  \
1         up to date  documentation + physical inspection   
0         up to date  documentation + physical inspection   
7         up to date  documentation + physical inspection   
5         up to date  documentation + physical inspection   
4         up to date  documentation + physical inspection   

  SPEC_incentive status  SPEC_Model year  SPEC_Production Year SPEC_Country  \
1               missing             2014                   NaN          USA   
0               missing             2014                   NaN          USA   
7               missing             2014                   NaN          USA   
5               missing             2014                   NaN          USA   
4               missing             2014                   NaN          USA   

  SPEC_Make SPEC_Global equivalent make SPEC_Model  \
1     ACURA                         NaN        MDX   
0     ACURA                         NaN        MDX   
7     ACURA                         NaN        MDX   
5     ACURA                         NaN        MDX   
4     ACURA                         NaN        MDX   

  SPEC_Global equivalent model SPEC_Model descriptor  SPEC_Model generation  \
1                          NaN                     -                    3.0   
0                          NaN                     -                    3.0   
7                          NaN                     -                    3.0   
5                          NaN                     -                    3.0   
4                          NaN                     -                    3.0   

                    SPEC_Version name  SPEC_Engine level SPEC_Special edition  \
1                        3.5 4WD AUTO                NaN                  NaN   
0                            3.5 AUTO                NaN                  NaN   
7  3.5 AUTO 4WD ADVANCE & ENT PACKAGE                NaN                  NaN   
5     3.5 AUTO 4WD TECH & ENT PACKAGE                NaN                  NaN   
4     3.5 AUTO 4WD TECHNOLOGY PACKAGE                NaN                  NaN   

  SPEC_Limited edition  SPEC_Special edition base UID  \
1                  NaN        

In [484]:
df_spec['duration'] = df_spec['end_date'] - df_spec['start_date']

In [485]:
df_spec['duration_lastupdate'] = timedelta(days=365) - df_spec.groupby('MY_Make_Model_Version')['duration'].transform('sum')

In [486]:
df_spec['duration'] = df_spec.apply(lambda x: x['duration'] if ｘ['duration'] >=timedelta(days=1) else x['duration_lastupdate'], axis=1)
df_spec['duration'] = df_spec['duration'].apply(lambda x: timedelta(days=0) if x < timedelta(days=0) else x)

In [487]:
df_spec['weight'] = df_spec['duration'] / df_spec.groupby(['MY_Make_Model_Version'])['duration'].transform('sum')

In [488]:
weighted_msrp = pd.DataFrame(df_spec.groupby('MY_Make_Model_Version').apply(lambda x: (x['weight'] * x['SPEC_Retail price']).sum()))
weighted_msrp.columns = ['Weighted_RetailPrice']
weighted_msrp.reset_index(inplace=True)

In [489]:
weighted_msrp

,MY_Make_Model_Version,Weighted_RetailPrice
0,2013 BMW X5 4.4 AUTO 4WD XDRIVE M,88850.0
1,2013 HONDA CIVIC 1.5 AUTO HYBRID CVT,24360.0
2,2013 HONDA CIVIC 1.5 AUTO HYBRID CVT W/LEATHER,25560.0
3,2013 HONDA CIVIC 1.5 AUTO HYBRID CVT W/LEATHER...,27060.0
4,2013 HONDA CIVIC 1.5 AUTO HYBRID CVT W/NAVI,25860.0
...,...,...
14262,2024 FORD E-SERIES 7.3 E-350 STRIPPED CHASSIS ...,34020.0
14263,2024 FORD E-SERIES 7.3 E-450 CUTAWAY 158-IN,38940.0
14264,2024 FORD E-SERIES 7.3 E-450 CUTAWAY 176-IN,39670.0
14265,2024 FORD E-SERIES 7.3 E-450 STRIPPED CHASSIS ...,35495.0


In [490]:
df_spec = pd.merge(df_spec, weighted_msrp, on='MY_Make_Model_Version', how='left')

In [491]:
dupli = df_spec[df_spec.duplicated(subset='MY_Make_Model_Version', keep=False)]

In [492]:
dupli = dupli[dupli['Volume']>0]

In [493]:
len(dupli)

31847

In [494]:
df_spec_wpricechange = df_spec.copy()

In [495]:
df_spec = df_spec_wpricechange[df_spec_wpricechange['SPEC_Version state']!='price change']

In [496]:
df_spec = df_spec[~df_spec.duplicated(subset=['MY_Make_Model_Version','SPEC_Transmission type',
                                   'SPEC_Version state', 'SPEC_Number of doors','SPEC_Body type',
                                   'SPEC_type','SPEC_Driven wheels', 'SPEC_Liters', 'SPEC_Retail price', 'SPEC_trim level11'])].reset_index(drop=True)

In [497]:
df_spec.drop(['MY_Make_Model_Version', 'start_date', 'end_date', 'duration',
       'duration_lastupdate', 'weight'],axis=1,inplace=True)

## NCBS 

In [498]:
df_ncbs = pd.read_excel('./Combination_Data/NCBS/NCBSFullmodel.xlsx')

In [499]:
df_ncbs['Make'] = df_ncbs['New Model Purchased - Make/Model/Series (Segment Order)'].apply(lambda x: x.split(' ')[0])

In [500]:
df_ncbs['Model'] = df_ncbs['New Model Purchased - Make/Model/Series (Segment Order)'].apply(lambda x: x.split(' ')[1])

In [501]:
df_ncbs['ModelYear']= df_ncbs['New Model Purchased - Make/Model/Series (Segment Order)'].apply(lambda x: re.search(r'\d\d\d\d MY', x)[0].replace(' MY', '')if len(x.split('('))>1 else 'NA')

In [502]:
df_ncbs =df_ncbs[df_ncbs['ModelYear']!='NA']

In [503]:
two_model = ['Bolt', 'City','Silverado','Mustang','Transit','Sierra','Ioniq','Santa', 'Rogue', 'Titan', 'Versa', 'Promaster', 'Model','Prius', 'Yaris']
four_model = ['Rover']

In [504]:
df_ncbs['Make'] = df_ncbs.apply(lambda x: ' '.join(x['New Model Purchased - Make/Model/Series (Segment Order)'].split(' ')[0:2] ) if x['Model'] in four_model else x['Make'], axis=1)

In [505]:
df_ncbs['Model'] = df_ncbs.apply(lambda x: ' '.join(x['New Model Purchased - Make/Model/Series (Segment Order)'].split(' ')[1:3] ) if x['Model'] in two_model else x['Model'], axis=1)
df_ncbs['Model'] = df_ncbs.apply(lambda x: ' '.join(x['New Model Purchased - Make/Model/Series (Segment Order)'].split(' ')[2:5] ) if x['Model'] in four_model else x['Model'], axis=1)
df_ncbs['Model'] = df_ncbs['Model'].str.replace('Rogue Platinum', 'Rogue').replace('Rogue S', 'Rogue').replace('Rogue SL', 'Rogue').replace('Rogue SV', 'Rogue')
df_ncbs['Model'] = df_ncbs['Model'].str.replace('Titan Crew', 'Titan').replace('Titan King', 'Titan').replace('Titan Platinum', 'Titan').replace('Titan PRO-4X', 'Titan')\
.replace('Titan S', 'Titan').replace('Titan SV', 'Titan')
df_ncbs['Model'] = df_ncbs['Model'].str.replace('Versa S', 'Versa').replace('Versa SL', 'Versa').replace('Versa SR', 'Versa').replace('Versa SV', 'Versa')
df_ncbs['Model'] = df_ncbs['Model'].str.replace('(', '').apply(lambda x: re.sub('Prius \d*', 'Prius', x)).str.replace('PriusAWD-e', 'Prius')
df_ncbs['Model'] = df_ncbs['Model'].str.replace('Yaris 2', 'Yaris').replace('Yaris 4', 'Yaris')

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/3025245572.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_ncbs['Model'] = df_ncbs['Model'].str.replace('(', '').apply(lambda x: re.sub('Prius \d*', 'Prius', x)).str.replace('PriusAWD-e', 'Prius')


In [506]:
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make'] + ' ' + df_ncbs['Model']

In [507]:
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW 3\S*', 'BMW Series3', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW M3\S*', 'BMW Series3', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW M3', 'BMW Series3', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW 4\S*', 'BMW Series4', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW M4\S*', 'BMW Series4', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW M4', 'BMW Series4', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW 5\S*', 'BMW Series5', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW M5', 'BMW Series5', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('BMW M5\S*', 'BMW Series5', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Ford E-\S*', 'Ford ESeries', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Mercedes-Benz C\d\S*', 'Mercedes-Benz C_Class', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Mercedes-Benz E\d\S*', 'Mercedes-Benz E_Class', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Mercedes-Benz GLA\S*', 'Mercedes-Benz GLA_Class', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Mercedes-Benz GLC\S*', 'Mercedes-Benz GLC_Class', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Mercedes-Benz GLE\S*', 'Mercedes-Benz GLE_Class', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Mercedes-Benz GLS\S*', 'Mercedes-Benz GLS_Class', x))
df_ncbs['Make_Model_ncbs'] = df_ncbs['Make_Model_ncbs'].apply(lambda x: re.sub('Ford Transit-\S*', 'Ford Transit', x))

In [508]:
df_ncbs['UID_ncbs'] = df_ncbs['ModelYear'] + ' ' + df_ncbs['Make_Model_ncbs']

In [509]:
for column in df_ncbs.columns:
    if 'OVERALL' in column:
        df_ncbs[column] = df_ncbs[column] * df_ncbs['Sample']

In [510]:
df_ncbs_gb = df_ncbs.groupby(['UID_ncbs']).sum().reset_index()

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/3467988051.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_ncbs_gb = df_ncbs.groupby(['UID_ncbs']).sum().reset_index()


In [511]:
for column in df_ncbs_gb.columns:
    if 'OVERALL' in column:
        df_ncbs_gb[column] = df_ncbs_gb[column] / df_ncbs_gb['Sample']

In [512]:
df_ncbs_gb

,UID_ncbs,Sample,OVERALL_Experience,OVERALL Value For The Money,OVERALL_Recommendation_Likeliness,OVERALL Exterior Styling,OVERALL Interior Styling/Aesthetics,OVERALL Environmental Friendliness,OVERALL Brand Image,OVERALL Comfort of the Seats,OVERALL Driver-Vehicle Interaction (HMI),OVERALL Driving Performance,OVERALL Noise/Vibration Harshness,OVERALL Performance of Climate Control System,OVERALL Power And Pickup,OVERALL Interior Versatility/Seat Configurability,OVERALL Interior Design / Layout,OVERALL Interior Workmanship,OVERALL_Interior Roominess,OVERALL Safety of the Vehicle,OVERALL Impression of Durability/Reliability,OVERALL Driver Assist Features
0,2012 Acura MDX,64,5.679215,7.172299,8.646154,7.546360,8.069561,6.429056,7.949326,8.115387,0.000000,7.991000,6.688649,7.977907,7.912441,7.729733,7.941900,7.930466,7.738955,7.712768,7.824506,0.000000
1,2012 Acura TL,64,6.205152,7.929495,9.158736,8.231430,8.399971,7.219577,8.247353,8.205692,0.000000,8.763866,7.743938,8.381638,8.762699,7.657412,8.410892,8.427048,7.784413,8.135726,8.185056,0.000000
2,2012 Acura TSX,144,6.165862,7.853851,8.970911,8.155860,7.894028,7.505830,8.467262,8.171708,0.000000,8.207370,7.587596,8.292161,7.597608,7.829168,8.147155,8.114091,7.429779,8.166362,8.087148,0.000000
3,2012 Acura ZDX,5,6.365801,6.470764,10.000000,8.265770,8.337284,3.988171,9.329175,8.973393,0.000000,9.329175,8.597574,5.075554,9.158350,9.158350,8.375485,9.500000,5.075554,9.500000,8.802568,0.000000
4,2012 Audi R8,4,7.000000,8.057471,9.500000,9.500000,9.028736,6.557471,9.500000,9.500000,0.000000,9.500000,9.500000,9.500000,9.500000,8.557471,9.500000,9.500000,8.086207,9.500000,9.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3568,2023 Volvo V60,3,6.666667,6.166667,9.333333,7.500000,7.500000,5.500000,7.500000,8.166667,6.833333,8.833333,6.166667,6.166667,8.166667,6.833333,8.833333,8.833333,8.166667,8.166667,7.500000,8.166667
3569,2023 Volvo V90,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3570,2023 Volvo XC40,50,5.495686,5.496860,8.331667,7.287804,7.026341,5.386000,7.186667,6.995000,5.836000,6.985244,5.740000,5.614545,6.206585,6.618000,6.961341,7.313171,6.286275,7.616279,6.580000,6.252727
3571,2023 Volvo XC60,132,5.487320,5.213860,8.236332,7.344076,6.853331,5.627059,6.862225,6.953464,5.579731,6.898119,6.035937,5.734058,6.637134,6.270132,6.500108,6.700868,5.888217,7.318182,6.590909,6.273855


## Join tables

In [513]:
spec_ncbs_jointable = pd.read_excel('./Combination_Data/NCBS/spec_ncbs_jointable.xlsx')

### Combination

In [514]:
spec_ncbs_jointable

,Make_Model_spec,Make_Model_ncbs
0,ACURA MDX,Acura MDX
1,ACURA NSX,Acura NSX
2,ACURA RDX,Acura RDX
3,ACURA RLX,Acura RLX
4,ACURA TLX,Acura TLX
...,...,...
159,TOYOTA TACOMA,Toyota Tacoma
160,TOYOTA TUNDRA,Toyota Tundra
161,TOYOTA YARIS,Toyota Yaris
162,TOYOTA YARIS IA,Toyota Yaris iA


In [515]:
df_spec['Make_Model_spec'] = df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model']

In [516]:
df_spec = pd.merge(df_spec, spec_ncbs_jointable, on='Make_Model_spec', how='left')

In [517]:
df_spec['NCBS_UID_ncbs'] = df_spec['SPEC_Model year'].astype(str) + ' ' + df_spec['Make_Model_ncbs'] 

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/903851407.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['NCBS_UID_ncbs'] = df_spec['SPEC_Model year'].astype(str) + ' ' + df_spec['Make_Model_ncbs']


In [518]:
df_ncbs_gb = df_ncbs_gb.add_prefix('NCBS_')

In [519]:
sum(df_ncbs_gb['NCBS_Sample'] < 10)

507

In [520]:
df_spec = pd.merge(df_spec, df_ncbs_gb, on='NCBS_UID_ncbs', how='left')

In [521]:
len(df_spec[df_spec['NCBS_OVERALL Noise/Vibration Harshness'].isna()])/len(df_spec)

0.10511017980262269

In [522]:
print(df_spec.columns.tolist())

['SPEC_Name', 'SPEC_Unique Identity', 'SPEC_Data date', 'SPEC_Date of change', 'SPEC_Conclude date', 'SPEC_Version state', 'SPEC_Data status', 'SPEC_Data completeness', 'SPEC_msrp', 'SPEC_colour & trim status', 'SPEC_key leasing items', 'SPEC_option content status', 'SPEC_option prices status', 'SPEC_prices status', 'SPEC_specs status', 'SPEC_incentive status', 'SPEC_Model year', 'SPEC_Production Year', 'SPEC_Country', 'SPEC_Make', 'SPEC_Global equivalent make', 'SPEC_Model', 'SPEC_Global equivalent model', 'SPEC_Model descriptor', 'SPEC_Model generation', 'SPEC_Version name', 'SPEC_Engine level', 'SPEC_Special edition', 'SPEC_Limited edition', 'SPEC_Special edition base UID', 'SPEC_Trim level', 'SPEC_trim classification', 'SPEC_trim code', 'SPEC_Pricing descriptor', 'SPEC_Transmission type', 'SPEC_Number of doors', 'SPEC_Body type', 'SPEC_cab type', 'SPEC_tilting cab', 'SPEC_body length', 'SPEC_duty rating', 'SPEC_Powertrain type', 'SPEC_msrp2', 'SPEC_type', 'SPEC_roof type', 'SPEC_wh

In [523]:
df_spec.drop(['Make_Model_spec', 'Make_Model_ncbs','NCBS_UID_ncbs'], axis=1, inplace=True)

# Inventory

In [524]:
df_spec['Make_Model_Bodytype'] = df_spec['SPEC_Make'] + ' ' + df_spec['SPEC_Model'] + ' ' + df_spec['SPEC_Body type'] + ' ' + df_spec['SPEC_Driven wheels']

In [525]:
df_inventory = pd.read_excel('./Combination_Data/Inventory/Inventory.xlsx')

In [526]:
inventory_jointable = pd.read_excel('./Combination_Data/Inventory/spec_inventory_jointable.xlsx')

### Combination

In [527]:
df_spec = pd.merge(df_spec, inventory_jointable, on='Make_Model_Bodytype', how='left')

In [528]:
inventory_jointable

,Make_Model_Bodytype,UID_Inventory
0,ACURA MDX sport utility vehicle 4x4,ACURA MDX
1,ACURA MDX sport utility vehicle front,ACURA MDX
2,ACURA NSX coupe 4x4,ACURA NSX
3,ACURA RDX sport utility vehicle front,ACURA RDX
4,ACURA RDX sport utility vehicle 4x4,ACURA RDX
...,...,...
337,TOYOTA YARIS hatchback front,TOYOTA YARIS
338,TOYOTA YARIS sedan front,TOYOTA YARIS
339,TOYOTA YARIS IA sedan front,TOYOTA YARIS iA/SCION iA
340,VOLKSWAGEN ID.4 sport utility vehicle rear,VOLKSWAGEN ID.4


In [529]:
df_spec['Inventory'] = np.nan
for i in range(len(df_spec)):
    UID_inventory = df_spec['UID_Inventory'][i]
    df_spec['Inventory'][i] = df_inventory[df_inventory['UID_Inventory'] == UID_inventory].reset_index(drop=True).loc[0,int(df_spec['SPEC_Data date'][i].strftime('%Y%m'))]

/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/3408519773.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_spec['Inventory'] = np.nan
/var/folders/57/0jgh1fz973nb1t0tp3f4y3b80000gn/T/ipykernel_27338/3408519773.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spec['Inventory'][i] = df_inventory[df_inventory['UID_Inventory'] == UID_inventory].reset_index(drop=True).loc[0,int(df_spec['SPEC_Data date'][i].strftime('%Y%m'))]


In [530]:
len(df_spec[df_spec['Inventory'].isna()])/len(df_spec)

0.04711369474111126

In [531]:
df_spec.drop(['Make_Model_Bodytype','UID_Inventory'], axis=1, inplace=True)

# Macro data

In [533]:
df_macro = pd.read_csv('./Combination_Data/Macro/merged_table_final_filtered_all_data.csv')

In [536]:
df_macro['DATE']

0      Jan-13
1      Feb-13
2      Mar-13
3      Apr-13
4      May-13
        ...  
118    Nov-22
119    Dec-22
120    Jan-23
121    Feb-23
122    Mar-23
Name: DATE, Length: 123, dtype: object

In [537]:
df_macro['DATE'] = pd.to_datetime(df_macro['DATE'], format="%b-%y")

In [541]:
df_macro['DATE'] = df_macro['DATE'].dt.strftime('%Y-%m')

In [542]:
df_spec.filter(like='Date')

,SPEC_Date of change,SPEC_Date version was linked,SPEC_Date introduced
0,2013-06-20,NaN,standard
1,2013-06-20,NaN,standard
2,2013-06-20,NaN,standard
3,2013-06-20,NaN,standard
4,2013-06-20,NaN,standard
...,...,...,...
14789,2022-08-05,NaN,standard
14790,2022-12-19,NaN,standard
14791,2022-08-05,NaN,standard
14792,2022-08-05,NaN,standard


In [543]:
df_spec['DATE'] = pd.to_datetime(df_spec['SPEC_Date of change']).dt.strftime('%Y-%m')

In [546]:
df_macro = df_macro.set_index('DATE').add_prefix('MACRO_').reset_index()

In [547]:
df_spec = pd.merge(df_spec, df_macro, on='DATE', how='left')

In [548]:
df_spec.to_csv('../01_Outputs/df_Preparation.csv', index=False)